# 유저 기반 협업 필터링

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 - 아이템 평점 메트릭스
ratings = np.array([
  [5,3,4,2,0,0,0], # 행 하나가 0번 사용자의 평점 리스트
  [4,3,3,2,3,2,1],
  [4,3,4,3,5,4,3],
  [3,3,1,5,4,2,3],
  [1,5,5,2,1,4,2]
])

In [2]:
# 코사인 유사도
user_similarity = cosine_similarity(ratings)
print("사용자간 유사도")
print(user_similarity)

사용자간 유사도
[[1.         0.84920778 0.69402209 0.60523674 0.6868296 ]
 [0.84920778 1.         0.95685784 0.8602264  0.81126307]
 [0.69402209 0.95685784 1.         0.90121683 0.82589664]
 [0.60523674 0.8602264  0.90121683 1.         0.68470256]
 [0.6868296  0.81126307 0.82589664 0.68470256 1.        ]]


In [3]:
# 피어슨 상관계수
# 전치행렬(.T)을 사용하는 이유:
# 원래 ratings 행렬은 (사용자 x 아이템) 형태입니다
# corr()은 행(row) 간의 상관관계를 계산하므로 -> 즉 (0,1) 과 (0,2) 의 상관관계를 구하는것 즉 열끼리 비교하기 위해 행 단위로 값을 본다. 
# 아이템 간의 상관관계를 구하려면 전치하여 (아이템 x 사용자) 형태로 변환해야 합니다
corrP = pd.DataFrame(ratings).T.corr() # 전치 행렬 
corrP


,0,1,2,3,4
0,1.000000,0.738371,-0.105915,-0.248069,0.180649
1,0.738371,1.000000,0.484123,-0.132288,-0.041286
2,-0.105915,0.484123,1.000000,-0.170783,-0.284268
3,-0.248069,-0.132288,-0.170783,1.000000,-0.655397
4,0.180649,-0.041286,-0.284268,-0.655397,1.000000


## 피어슨 vs 코사인
- 피어슨상관 계수 : 두데이터의 변화 경향이 얼마나 비슷한지
  - 단순 값의 크기 보다는 증가/감속하는 패턴에 초점
- 코사인 유사도 : 두 벡터가 얼마나 비슷한 방향을 가지고 있는지


In [4]:
# 0번 사용자의 평가 안한 아이템 평점 예측
similar_users = user_similarity[0]
similar_users_P = corrP[0]

print(similar_users)
print(similar_users_P)

# 0번 사용자의 평가 안한 아이템 평점 예측
# 코사인 유사도



[1.         0.84920778 0.69402209 0.60523674 0.6868296 ]
0    1.000000
1    0.738371
2   -0.105915
3   -0.248069
4    0.180649
Name: 0, dtype: float64


In [5]:
# 영화 5번에 대한 예측 평점

# 영화 5번 평점
print(ratings[:,4]) # [0 3 5 4 1]
print(similar_users) # [1.         0.84920778 0.69402209 0.60523674 0.6868296 ]
print(similar_users)

# 가중합
# 즉 (0*1 + 3*0.84920778 + 5*0.69402209 + 4*0.60523674 + 1*0.6868296) / (0.84920778 + 0.69402209 + 0.60523674 + 0.6868296) = 3.2185386196

[0 3 5 4 1]
[1.         0.84920778 0.69402209 0.60523674 0.6868296 ]
[1.         0.84920778 0.69402209 0.60523674 0.6868296 ]


In [8]:
def get_ubcf_score(ind):
  predicted_rating_for_movie_5 = np.dot(similar_users, ratings[:,ind]) / np.sum(similar_users) # 여기에 자기 자긴의 유사도인 1.0이 들어가서 계산이 안맞음 이거 제거되어야함
  print(f'영화 {ind+1}에 대한 사용자 A의 예측 평점: {predicted_rating_for_movie_5}')

In [9]:
get_ubcf_score(4)
get_ubcf_score(5)
get_ubcf_score(6)


영화 5에 대한 사용자 A의 예측 평점: 2.3793495656260024
영화 6에 대한 사용자 A의 예측 평점: 2.1986035331720233
영화 7에 대한 사용자 A의 예측 평점: 1.5958724299991014


# 아이템 기반 협업 필터링
- 방식은 거이 유사
- 둘 방식간 예측값은 달라져
- 두 방식을 하이브리드로 사용 가능

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 - 아이템 평점 메트릭스
ratings = np.array([
  [5,3,4,2,0,0,0], # 행 하나가 0번 사용자의 평점 리스트
  [4,3,3,2,3,2,1],
  [4,3,4,3,5,4,3],
  [3,3,1,5,4,2,3],
  [1,5,5,2,1,4,2]
])

In [10]:
item_similarity = cosine_similarity(ratings.T)
print("아이템간 유사도")
print(item_similarity)


아이템간 유사도
[[1.         0.82903631 0.8358209  0.84660639 0.76982148 0.6567673
  0.68780043]
 [0.82903631 1.         0.95417386 0.86838837 0.73507872 0.89075396
  0.82762358]
 [0.8358209  0.95417386 1.         0.73852898 0.65007147 0.84993415
  0.71327452]
 [0.84660639 0.86838837 0.73852898 1.         0.88777815 0.79262909
  0.92231319]
 [0.76982148 0.73507872 0.65007147 0.88777815 1.         0.84133414
  0.93433146]
 [0.6567673  0.89075396 0.84993415 0.79262909 0.84133414 1.
  0.92313266]
 [0.68780043 0.82762358 0.71327452 0.92231319 0.93433146 0.92313266
  1.        ]]


In [11]:
user_0_ratings = ratings[0]
def get_ibcf_score(ind):
  predicted_rating_for_movie_5 = np.dot(user_0_ratings, item_similarity[:, ind]) / np.sum(item_similarity[:,ind])
  print(f'영화 {ind+1}에 대한 사용자 A의 예측 평점: {predicted_rating_for_movie_5}')
get_ibcf_score(4)
get_ibcf_score(5)
get_ibcf_score(6)



영화 5에 대한 사용자 A의 예측 평점: 1.7926161998371786
영화 6에 대한 사용자 A의 예측 평점: 1.8374336870101942
영화 7에 대한 사용자 A의 예측 평점: 1.7674361436289245
